In [1]:
import pandas as pd
# 读取CSV文件
df = pd.read_csv("data/train.csv")

In [2]:
df.head()

,id,heartbeat_signals,label
0,0,"0.9912297987616655,0.9435330436439665,0.764677...",0.0
1,1,"0.9714822034884503,0.9289687459588268,0.572932...",0.0
2,2,"1.0,0.9591487564065292,0.7013782792997189,0.23...",2.0
3,3,"0.9757952826275774,0.9340884687738161,0.659636...",0.0
4,4,"0.0,0.055816398940721094,0.26129357194994196,0...",2.0


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import xgboost as xgb

 

# 步骤1: 数据预处理
# 分割心跳信号序列为数值列表
df['heartbeat_signals'] = df['heartbeat_signals'].apply(lambda x: [float(i) for i in x.split(',')])

# 将列表转换为DataFrame的列
max_length = max(df['heartbeat_signals'].apply(len))  # 找到最长序列的长度
signals_df = pd.DataFrame(df['heartbeat_signals'].tolist()).fillna(0)  # 用0填充缺失值
df = pd.concat([df, signals_df], axis=1)

# 删除原始的heartbeat_signals列
df = df.drop('heartbeat_signals', axis=1)

ModuleNotFoundError: No module named 'xgboost'

In [4]:
pip install xgboost

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.1/297.1 MB 125.8 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import xgboost as xgb

 

# 步骤1: 数据预处理
# 分割心跳信号序列为数值列表
df['heartbeat_signals'] = df['heartbeat_signals'].apply(lambda x: [float(i) for i in x.split(',')])

# 将列表转换为DataFrame的列
max_length = max(df['heartbeat_signals'].apply(len))  # 找到最长序列的长度
signals_df = pd.DataFrame(df['heartbeat_signals'].tolist()).fillna(0)  # 用0填充缺失值
df = pd.concat([df, signals_df], axis=1)

# 删除原始的heartbeat_signals列
df = df.drop('heartbeat_signals', axis=1)

In [6]:
  df.head()

,id,label,0,1,2,3,4,5,6,7,...,195,196,197,198,199,200,201,202,203,204
0,0,0.0,0.991230,0.943533,0.764677,0.618571,0.379632,0.190822,0.040237,0.025995,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.971482,0.928969,0.572933,0.178457,0.122962,0.132360,0.094392,0.089575,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,2.0,1.000000,0.959149,0.701378,0.231778,0.000000,0.080698,0.128376,0.187448,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0.0,0.975795,0.934088,0.659637,0.249921,0.237116,0.281445,0.249921,0.249921,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,2.0,0.000000,0.055816,0.261294,0.359847,0.433143,0.453698,0.499004,0.542796,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
pip install imblearn

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.7/257.7 kB 119.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [8]:
from imblearn.over_sampling import SMOTE
# 步骤3: 模型训练
X = df.drop(['id', 'label'], axis=1)
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# 使用SMOTE进行过采样处理类别不平衡，只对训练集进行处理
smote = SMOTE()
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

In [9]:
from collections import Counter

# 查看过采样后的类别分布
class_distribution = Counter(y_resampled)
print(class_distribution)


Counter({0.0: 51402, 2.0: 51402, 3.0: 51402, 1.0: 51402})


In [10]:
from scipy.stats import randint, uniform

param_dist = {
    'n_estimators': randint(100, 1000),
    'learning_rate': uniform(0.01, 0.3),
    'subsample': uniform(0.7, 0.3),
    'max_depth': randint(3, 10),
    'colsample_bytree': uniform(0.7, 0.3),
    'min_child_weight': randint(1, 10)
}


In [11]:
from sklearn.model_selection import RandomizedSearchCV

# 初始化XGBClassifier
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

# 设置随机搜索
random_search = RandomizedSearchCV(model, param_distributions=param_dist, n_iter=25, scoring='neg_log_loss', cv=5, verbose=3, random_state=42)

# 使用全部数据进行拟合
random_search.fit(X_resampled, y_resampled)


Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END colsample_bytree=0.8123620356542087, learning_rate=0.2952142919229748, max_depth=5, min_child_weight=8, n_estimators=800, subsample=0.879055047383946;, score=-0.018 total time= 1.8min
[CV 2/5] END colsample_bytree=0.8123620356542087, learning_rate=0.2952142919229748, max_depth=5, min_child_weight=8, n_estimators=800, subsample=0.879055047383946;, score=-0.010 total time= 1.7min
[CV 3/5] END colsample_bytree=0.8123620356542087, learning_rate=0.2952142919229748, max_depth=5, min_child_weight=8, n_estimators=800, subsample=0.879055047383946;, score=-0.009 total time= 1.7min
[CV 4/5] END colsample_bytree=0.8123620356542087, learning_rate=0.2952142919229748, max_depth=5, min_child_weight=8, n_estimators=800, subsample=0.879055047383946;, score=-0.008 total time= 1.7min
[CV 5/5] END colsample_bytree=0.8123620356542087, learning_rate=0.2952142919229748, max_depth=5, min_child_weight=8, n_estimators=800, subsample=0.879

KeyboardInterrupt: 

In [14]:
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss
import numpy as np
import xgboost as xgb

kf = KFold(n_splits=5, shuffle=True, random_state=42)

abs_sum_scores = []  # 存储每一折的评分

for train_index, test_index in kf.split(X):
    X_resampled, X_test = X.iloc[train_index], X.iloc[test_index]
    y_resampled, y_test = y[train_index], y[test_index]
    # 使用找到的最佳参数设置模型
    best_params = {
    'colsample_bytree': 0.7692681476866446,
    'learning_rate': 0.0823076398078035,
    'max_depth': 6,
    'min_child_weight': 7,
    'n_estimators': 527,
    'subsample': 0.848553073033381,
    'use_label_encoder': False,
    'eval_metric': 'mlogloss'
   }

    # 初始化XGBoost模型
    model = xgb.XGBClassifier(**best_params)
    
    model.fit(X_resampled, y_resampled)
    
    # 预测概率
    y_pred_proba = model.predict_proba(X_test)
    
    # 计算abs-sum
    # 首先，我们需要将y_test转换为one-hot编码形式，以匹配y_pred_proba的格式
    y_test_one_hot = np.zeros((y_test.size, y_pred_proba.shape[1]))
    y_test_one_hot[np.arange(y_test.size), y_test.astype(int)] = 1
    
    abs_sum = np.abs(y_test_one_hot - y_pred_proba).sum() / y_test.size
    abs_sum_scores.append(abs_sum)

# 计算平均abs-sum分数
average_abs_sum = np.mean(abs_sum_scores)
print(f"Average Abs-Sum Score: {average_abs_sum}")


KeyboardInterrupt: 